This script builds a predictive model using XGBoost to identify users who are likely to withdraw their credits 
(based on behavioral and demographic data). It performs the following steps:

1. **Data Loading**: Loads three datasets: user demographic features (Atlas), payment transactions, and current credit balances.

2. **Cleaning**: Filters out invalid rows (e.g., zero credits, missing user IDs).

3. **Feature Engineering (Atlas)**:
   - Transforms binary-encoded demographic features grouped by category.
   - Encodes categorical sets into numeric IDs for model compatibility.

4. **Withdrawal Stats Aggregation**:
   - Calculates historical withdrawal frequency and volume per user.

5. **Target Construction**:
   - Merges current eligible users (credits ≥ 500) with past withdrawal stats.
   - Assigns a binary `target` indicating whether a user is likely to withdraw now, based on average past behavior.

6. **Data Merging**:
   - Combines processed Atlas features with credit and withdrawal stats into one modeling dataframe.

7. **Preprocessing**:
   - Drops irrelevant or leakage-prone features.
   - Scales numerical data using `StandardScaler`.

8. **Modeling**:
   - Splits data into training/testing sets with stratification.
   - Defines a hyperparameter grid and performs cross-validated training (Stratified K-Fold, n=5).
   - Tunes classification threshold to maximize F1-score for class 1 (users likely to withdraw).
   - Tracks and logs the best performing model and evaluation report.

9. **Logging**:
   - Outputs detailed logs at each major step including data loading, feature processing, training, and evaluation.

In [1]:
import logging
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import defaultdict
import xgboost as xgb
from sklearn.model_selection import ParameterSampler, StratifiedKFold
from sklearn.metrics import classification_report, f1_score
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import joblib
import os

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Load datasets
logger.info("Loading datasets")
df_atlas = pd.read_csv('../data/Atlas Cechu Student Access.csv', encoding='utf-8')
df_payments = pd.read_csv('../data/Payments Student Access.csv', encoding='utf-8')
df_credits = pd.read_csv('../data/User Credits Student Access.csv', encoding='utf-8')

# Clean data
logger.info("Cleaning data")
df_credits_cleaned = df_credits[df_credits['credits'] > 0]
df_payments_cleaned = df_payments[df_payments['user'].notna()]

# Process Atlas data into structured format
logger.info("Transforming Atlas data")
grouped_cols = defaultdict(dict)
for col in df_atlas.columns:
    if "-" in col:
        group, key = col.split('-', 1)
        grouped_cols[group][key] = col
    else:
        grouped_cols[col][col] = col

structured_data = []
for _, row in df_atlas.iterrows():
    entry = {}
    for group, mapping in grouped_cols.items():
        entry[group] = [key for key, col in mapping.items() if row[col] == 1]
    structured_data.append(entry)

# Encode structured features
df_atlas_numeric_values = pd.DataFrame(structured_data).drop(columns=['user_id'])
df_atlas_nv = pd.concat([df_atlas['user_id'], df_atlas_numeric_values], axis=1, join='inner')
mapping_dicts = {}
for col in df_atlas_nv.columns:
    if col == 'user_id':
        continue
    unique_lists = df_atlas_nv[col].apply(lambda x: tuple(sorted(x))).unique()
    mapping_dicts[col] = {lst: idx + 1 for idx, lst in enumerate(unique_lists)}
    df_atlas_nv[col] = df_atlas_nv[col].apply(lambda x: mapping_dicts[col][tuple(sorted(x))])

# Extract withdrawal statistics
logger.info("Aggregating withdrawal stats")
withdrawals = df_payments[(df_payments['credits'] >= 500) & (df_payments['state'].isin(['PAID', 'APPROVED']))]
withdrawal_stats = withdrawals.groupby('user').agg(
    num_withdrawals=('credits', 'count'),
    avg_withdrawal=('credits', 'mean'),
    total_withdrawn=('credits', 'sum')
).reset_index()

# Identify currently eligible users
logger.info("Building eligible user set")
eligible_now = df_credits[df_credits['credits'] >= 500].copy()
eligible_now = eligible_now.merge(withdrawal_stats, on='user', how='left')
eligible_now[['num_withdrawals', 'avg_withdrawal', 'total_withdrawn']] = eligible_now[
    ['num_withdrawals', 'avg_withdrawal', 'total_withdrawn']
].fillna(0)

def assign_behavior(row):
    if row['num_withdrawals'] == 0:
        return 'new'
    elif row['num_withdrawals'] <= 2:
        return 'occasional'
    else:
        return 'regular'

eligible_now['withdrawal_segment'] = eligible_now.apply(assign_behavior, axis=1)
segment_mapping = {'new': 0, 'occasional': 1, 'regular': 2}
eligible_now['withdrawal_segment_code'] = eligible_now['withdrawal_segment'].map(segment_mapping)
eligible_now.drop(columns=['withdrawal_segment', 'wage'], inplace=True, errors='ignore')

def likely_to_withdraw_now(row):
    if row['num_withdrawals'] == 0:
        return 0
    expected_threshold = max(0.9 * row['avg_withdrawal'], 450)
    return int(row['credits'] >= expected_threshold)

eligible_now['target'] = eligible_now.apply(likely_to_withdraw_now, axis=1)

# Merge Atlas features
logger.info("Merging structured Atlas features")
df_atlas_clean_nv = df_atlas_nv.copy()
df_atlas_clean_nv.rename(columns={'user_id': 'user'}, inplace=True)
df_model = pd.merge(eligible_now, df_atlas_clean_nv, on='user', how='left')

# Feature engineering
X = df_model.drop(columns=[
    'target', 'user', 'credits',
    'num_withdrawals', 'avg_withdrawal', 'total_withdrawn', 
    'is_active', 'is_verified', 'is_locked'
])
y = df_model['target']
X = X.fillna(0)

# Normalize features
logger.info("Scaling features")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data for validation
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, test_size=0.2, random_state=42)
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_test, label=y_test)

# Set up cross-validation and hyperparameter tuning
logger.info("Starting cross-validated hyperparameter search")
n_splits = 5
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

best_f1 = 0
best_threshold = 0.5
best_params = None
best_booster = None
best_report = ""

param_dist = {
    'learning_rate': [0.1, 0.15, 0.2, 0.25],
    'max_depth': [6, 7, 8],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.9, 1.0],
    'scale_pos_weight': [1.0, 1.5, 2.0],
}
param_list = list(ParameterSampler(param_dist, n_iter=200, random_state=42))

def f1_eval(preds, dtrain):
    labels = dtrain.get_label()
    preds_binary = (preds > 0.5).astype(int)
    return 'f1', f1_score(labels, preds_binary)

# Run training loop with cross-validation
for params in tqdm(param_list, desc="CV Search"):
    full_params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'seed': 42,
        **params
    }
    cv_f1_scores = []

    for train_idx, valid_idx in cv.split(X_scaled, y):
        X_train_cv, X_valid_cv = X_scaled[train_idx], X_scaled[valid_idx]
        y_train_cv, y_valid_cv = y.iloc[train_idx], y.iloc[valid_idx]

        dtrain_cv = xgb.DMatrix(X_train_cv, label=y_train_cv)
        dvalid_cv = xgb.DMatrix(X_valid_cv, label=y_valid_cv)

        booster = xgb.train(
            full_params,
            dtrain_cv,
            num_boost_round=200,
            evals=[(dvalid_cv, 'eval')],
            early_stopping_rounds=10,
            verbose_eval=False
        )

        y_proba = booster.predict(dvalid_cv)

        # Threshold tuning
        best_f1_cv = 0
        for thresh in np.linspace(0.3, 0.55, 30):
            y_pred_cv = (y_proba > thresh).astype(int)
            f1_cv = f1_score(y_valid_cv, y_pred_cv)
            best_f1_cv = max(best_f1_cv, f1_cv)

        cv_f1_scores.append(best_f1_cv)

    mean_f1 = np.mean(cv_f1_scores)
    logger.info(f"Params: {params}, Mean F1: {mean_f1:.4f}")

    if mean_f1 > best_f1:
        best_f1 = mean_f1
        best_params = full_params
        best_booster = booster
        best_threshold = thresh
        y_final_pred = (y_proba > best_threshold).astype(int)
        best_report = classification_report(y_valid_cv, y_final_pred, output_dict=False)

logger.info(f"Best Mean F1: {best_f1:.4f} at threshold={best_threshold:.2f}")
logger.info(f"Best Params: {best_params}")
logger.info(f"Best CV Fold Report:\n{best_report}")

os.makedirs("models", exist_ok=True)

model_path = "models/best_xgb_model.json"
best_booster.save_model(model_path)

joblib.dump(scaler, "models/standard_scaler.pkl")
joblib.dump(best_threshold, "models/best_threshold.pkl")
joblib.dump(X.columns.tolist(), "models/feature_names.pkl")

logger.info(f"Model, scaler, and threshold saved to 'models/'")

2025-06-03 16:08:06,699 - INFO - Loading datasets
2025-06-03 16:08:08,555 - INFO - Cleaning data
2025-06-03 16:08:08,565 - INFO - Transforming Atlas data
2025-06-03 16:09:54,805 - INFO - Aggregating withdrawal stats
2025-06-03 16:09:54,838 - INFO - Building eligible user set
2025-06-03 16:09:54,960 - INFO - Merging structured Atlas features
2025-06-03 16:09:55,166 - INFO - Scaling features
2025-06-03 16:09:55,237 - INFO - Starting cross-validated hyperparameter search
CV Search: 100%|██████████| 200/200 [04:20<00:00,  1.30s/it]
2025-06-03 16:14:15,895 - INFO - Best Mean F1: 0.7294 at threshold=0.55
2025-06-03 16:14:15,896 - INFO - Best Params: {'objective': 'binary:logistic', 'eval_metric': 'logloss', 'seed': 42, 'subsample': 1.0, 'scale_pos_weight': 2.0, 'max_depth': 7, 'learning_rate': 0.15, 'colsample_bytree': 1.0}
2025-06-03 16:14:15,897 - INFO - Best CV Fold Report:
              precision    recall  f1-score   support

           0       0.96      0.85      0.90       559
       

In [2]:
# Prepare prediction input (must match training features)
X_pred = df_model.drop(columns=[
    'target', 'user', 'credits',
    'num_withdrawals', 'avg_withdrawal', 'total_withdrawn',
    'is_active', 'is_verified', 'is_locked'
], errors='ignore')

X_pred = X_pred.fillna(0)

# Ensure same scaling and order
X_pred_scaled = scaler.transform(X_pred)
dpred = xgb.DMatrix(X_pred_scaled)

# Predict using best booster and threshold
y_pred_proba_final = best_booster.predict(dpred)
df_model['prediction'] = (y_pred_proba_final > best_threshold).astype(int)

# Estimate total withdrawal amount
predicted_cashouts = df_model[df_model['prediction'] == 1]
total_predicted_credits = predicted_cashouts['credits'].sum()

# Count users predicted to withdraw
num_predicted_users = df_model['prediction'].sum()
print(f"👥 Estimated number of users who will cash out: {num_predicted_users}")

print(f"\n💰 Estimated reserve needed for upcoming withdrawals: {total_predicted_credits:,.0f} CZK")

👥 Estimated number of users who will cash out: 818

💰 Estimated reserve needed for upcoming withdrawals: 1,030,264 CZK


In [4]:
df_payments['created_at'] = pd.to_datetime(df_payments['created_at'], errors='coerce')

df_successful = df_payments[df_payments['state'].isin(['PAID', 'APPROVED'])].copy()

df_successful['year'] = df_successful['created_at'].dt.year
df_successful['month'] = df_successful['created_at'].dt.month

monthly_withdrawals = (
    df_successful.groupby(['year', 'month'])['credits']
    .sum()
    .reset_index()
    .sort_values(['year', 'month'])
)

import plotly.express as px
fig = px.bar(
    monthly_withdrawals,
    x='month',
    y='credits',
    color='year',
    barmode='group',
    title='Total Withdrawals per Month by Year',
    labels={'credits': 'Withdrawn Credits (CZK)', 'month': 'Month'}
)
fig.show()

import plotly.graph_objects as go
fig_table = go.Figure(data=[go.Table(
    header=dict(values=["Year", "Month", "Withdrawn Credits"]),
    cells=dict(values=[
        monthly_withdrawals['year'],
        monthly_withdrawals['month'],
        monthly_withdrawals['credits'].round(2)
    ])
)])
fig_table.show()


In [5]:
# Count number of withdrawals per month (state must be PAID or APPROVED)
monthly_withdrawal_counts = (
    df_successful
    .groupby(['year', 'month'])
    .size()
    .reset_index(name='withdrawal_count')
    .sort_values(['year', 'month'])
)


In [7]:
fig_count = px.bar(
    monthly_withdrawal_counts,
    x='month',
    y='withdrawal_count',
    color='year',
    barmode='group',
    title='Number of Successful Withdrawals per Month by Year',
    labels={'withdrawal_count': 'Number of Withdrawals', 'month': 'Month'}
)
fig_count.show()

fig_table_counts = go.Figure(data=[go.Table(
    header=dict(values=["Year", "Month", "Number of Withdrawals"]),
    cells=dict(values=[
        monthly_withdrawal_counts['year'],
        monthly_withdrawal_counts['month'],
        monthly_withdrawal_counts['withdrawal_count']
    ])
)])
fig_table_counts.show()